In [1]:
from os import listdir
from pickle import dump
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.utils import to_categorical
from nltk.translate.bleu_score import corpus_bleu
from pickle import load
from keras.optimizers import Adam

import matplotlib.pyplot as plt
import tensorflow as tf

import os
from PIL import Image

from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding, Dropout, LSTM, add, Reshape, Lambda
from tensorflow.python.keras.applications import VGG16
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.applications.vgg16 import decode_predictions
from tqdm import tqdm
import cv2
import PIL
import numpy as np
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K
from numpy import newaxis

Using TensorFlow backend.


In [2]:
features = load(open('features.pkl', 'rb'))

In [3]:
len(features)

5000

In [4]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [5]:
filename = "image_names.txt"
dataset = load_doc(filename)

In [6]:
def train_dev_seperation(doc):
    train_images=list()
    dev_images=list()
    i=0
    for line in doc.split('\n'):
        if i<50:
            train_images.append(line)
        elif i<60:
            dev_images.append(line)
        else:
            break
        i=i+1;
    return train_images, dev_images

In [7]:
train_names, dev_names = train_dev_seperation(dataset)

In [8]:
def save_images(filename, doc):
    data = '\n'.join(doc)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [9]:

save_images('train_images.txt', train_names)
save_images('dev_images.txt', dev_names)

In [10]:
def load_clean_descriptions(filename, dataset):
    # load document
    doc = load_doc(filename)
    descriptions = dict()
    for line in doc.split('\n'):
        # split line by white space
        tokens = line.split()
        # split id from description
        image_id, image_desc = tokens[0], tokens[1:]
        # skip images not in the set
        if image_id in dataset:
            # create list
            if image_id not in descriptions:
                descriptions[image_id] = list()
            # wrap description in tokens
            desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
            # store
            descriptions[image_id].append(desc)
    return descriptions

In [11]:
def load_photo_features(filename, dataset):
    # load all features
    all_features = load(open(filename, 'rb'))
    # filter features
    features = {k: all_features[k] for k in dataset}
    return features

In [12]:
# load a pre-defined list of photo identifiers
def load_set(filename):
    doc = load_doc(filename)
    dataset = list()
    # process line by line
    for line in doc.split('\n'):
        # skip empty lines
        if len(line) < 1:
            continue
        # get the image identifier
        identifier = line.split('.')[0]
        dataset.append(identifier)
    return set(dataset)

In [13]:
# load training dataset (6K)
filename = 'train_images.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))
# photo features
train_features = load_photo_features('features.pkl', train)
print('Photos: train=%d' % len(train_features))

Dataset: 50
Descriptions: train=50
Photos: train=50


In [14]:
print(len(train_features))
print(len(train_descriptions))
train_features["1000092795"].shape
train_descriptions["1000092795"]

50
50


['startseq two young guys with shaggy hair look at their hands while hanging out in the yard endseq',
 'startseq two young white males are outside near many bushes endseq',
 'startseq two men in green shirts are standing in yard endseq',
 'startseq man in blue shirt standing in garden endseq',
 'startseq two friends enjoy time spent together endseq']

In [15]:
# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
    all_desc = list()
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc
 
# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
    lines = to_lines(descriptions)
    print(len(lines))
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [16]:
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
word_to_seq = tokenizer.word_index
vocab_size = len(word_to_seq) + 1
print('Vocabulary Size: %d' % vocab_size)

250
Vocabulary Size: 678


In [56]:
train_features["1000092795"]

array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [18]:
def convert_to_one_hot(Y, C):
    Y = np.array(Y)
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

In [19]:
def max_length(descriptions):
    lines = to_lines(descriptions)
    return max(len(d.split()) for d in lines)

In [90]:
def create_sequences(tokenizer, max_length, descriptions, photos, vocab_size):
    a0, X, y = list(), list(), list()
    # walk through each image identifier
    for key, desc_list in descriptions.items():
        # walk through each description for the image
        for desc in desc_list:
            # encode the sequence
            seq = tokenizer.texts_to_sequences([desc])[0]
            seq = pad_sequences([seq], maxlen=max_length)[0]
            inp =convert_to_one_hot(seq[:-1], vocab_size)
            out =convert_to_one_hot(seq[1:], vocab_size)
            a0.append(photos[key][0])
            X.append(inp)
            y.append(out)
    return np.array(a0), np.array(X), np.array(y)

In [91]:
a0_train, X_train, Y_train = create_sequences(tokenizer, max_length(train_descriptions), train_descriptions, train_features, vocab_size)

In [92]:
a0_train.shape

(250, 4096)

In [93]:
X = X_train
Y = Y_train.reshape(34,250,678)

In [94]:
n_a = 128
n_values = vocab_size

In [108]:
def one_hot(x):
    x = K.argmax(x)
    x = tf.one_hot(x, 678) 
    x = RepeatVector(1)(x)
    return x

In [95]:
reshapor = Reshape((1, n_values))                        
LSTM_cell = LSTM(n_a, return_state = True)         
densor = Dense(n_values, activation='softmax')
transfer_densor = Dense(n_a, activation='tanh')

In [96]:
def djmodel(Tx, n_a, n_values):
    

    X = Input(shape=(Tx, n_values))
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(4096,))
    a = transfer_densor(a0)
    c= a
    # Step 2: Loop
    outputs = []
    for t in range(Tx):
        
        # Step 2.A: select the "t"th time step vector from X. 
        x = Lambda(lambda x: X[:,t,:])(X)
        # Step 2.B: Use reshapor to reshape x to be (1, n_values) (≈1 line)
        x = reshapor(x)
        # Step 2.C: Perform one step of the LSTM_cell
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        # Step 2.D: Apply densor to the hidden state output of LSTM_Cell
        out = densor(a)
        # Step 2.E: add the output to "outputs"
        outputs.append(out)
        
    # Step 3: Create model instance
    model = Model([X, a0], outputs)
    
    
    return model

In [97]:
model = djmodel(Tx = 34, n_a = 128, n_values = vocab_size)

In [98]:
filepath = 'checkpoints/model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [99]:
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [100]:
a0 = a0_train

In [101]:
model.fit([X, a0], list(Y), epochs=2)

Epoch 1/2
250/250 [==============================] - ETA: 10:28 - loss: 221.3340 - dense_3_loss: 6.5157 - dense_3_acc: 0.0000e+00 - dense_3_acc_1: 0.0000e+00 - dense_3_acc_2: 0.0000e+00 - dense_3_acc_3: 0.0000e+00 - dense_3_acc_4: 0.0000e+00 - dense_3_acc_5: 0.0000e+00 - dense_3_acc_6: 0.0000e+00 - dense_3_acc_7: 0.0000e+00 - dense_3_acc_8: 0.0000e+00 - dense_3_acc_9: 0.0000e+00 - dense_3_acc_10: 0.0000e+00 - dense_3_acc_11: 0.0000e+00 - dense_3_acc_12: 0.0000e+00 - dense_3_acc_13: 0.0000e+00 - dense_3_acc_14: 0.0000e+00 - dense_3_acc_15: 0.0000e+00 - dense_3_acc_16: 0.0000e+00 - dense_3_acc_17: 0.0000e+00 - dense_3_acc_18: 0.0000e+00 - dense_3_acc_19: 0.0000e+00 - dense_3_acc_20: 0.0000e+00 - dense_3_acc_21: 0.0000e+00 - dense_3_acc_22: 0.0000e+00 - dense_3_acc_23: 0.0000e+00 - dense_3_acc_24: 0.0000e+00 - dense_3_acc_25: 0.0000e+00 - dense_3_acc_26: 0.0000e+00 - dense_3_acc_27: 0.0000e+00 - dense_3_acc_28: 0.0000e+00 - dense_3_acc_29: 0.0000e+00 - dense_3_acc_30: 0.0000e+00 - dense_3

250/250 [==============================] - ETA: 11s - loss: 97.2539 - dense_3_loss: 4.1135 - dense_3_acc: 0.6875 - dense_3_acc_1: 0.7188 - dense_3_acc_2: 0.6562 - dense_3_acc_3: 0.6875 - dense_3_acc_4: 0.7188 - dense_3_acc_5: 0.5938 - dense_3_acc_6: 0.7500 - dense_3_acc_7: 0.5938 - dense_3_acc_8: 0.6250 - dense_3_acc_9: 0.7188 - dense_3_acc_10: 0.4375 - dense_3_acc_11: 0.7500 - dense_3_acc_12: 0.8125 - dense_3_acc_13: 0.3750 - dense_3_acc_14: 0.7500 - dense_3_acc_15: 0.6250 - dense_3_acc_16: 0.5312 - dense_3_acc_17: 0.6875 - dense_3_acc_18: 0.5938 - dense_3_acc_19: 0.5312 - dense_3_acc_20: 0.5625 - dense_3_acc_21: 0.6562 - dense_3_acc_22: 0.6875 - dense_3_acc_23: 0.7188 - dense_3_acc_24: 0.5312 - dense_3_acc_25: 0.6250 - dense_3_acc_26: 0.7188 - dense_3_acc_27: 0.6250 - dense_3_acc_28: 0.6875 - dense_3_acc_29: 0.8438 - dense_3_acc_30: 0.4688 - dense_3_acc_31: 0.7188 - dense_3_acc_32: 0.5625 - dense_3_acc_33: 0.46 - ETA: 7s - loss: 92.5208 - dense_3_loss: 3.6843 - dense_3_acc: 0.6406 - 

In [111]:
def inference_model(LSTM_cell, densor, transfer_densor, Ty = 34, n_a = 128, n_values = vocab_size):

    # Define the input of your model with a shape 
    x0 = Input(shape=(1, n_values))
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(4096,))
    a = transfer_densor(a0)
    c = a
    x = x0
    # Step 2: Loop
    outputs = []
    
    # Step 2: Loop over Ty and generate a value at every time step
    for i in range(Ty):
        
        # Step 2.A: Perform one step of LSTM_cell (≈1 line)
        a, _, c = LSTM_cell(x, initial_state = [a, c])
        
        # Step 2.B: Apply Dense layer to the hidden state output of the LSTM_cell (≈1 line)
        out = densor(a)

        # Step 2.C: Append the prediction "out" to "outputs". out.shape = (None, 78) (≈1 line)
        out = np.argmax(out)
        
        outputs.append(out)
        if np.argmax(out) == 2:
            break
        # Step 2.D: Select the next value according to "out", and set "x" to be the one-hot representation of the
        #           selected value, which will be passed as the input to LSTM_cell on the next step. We have provided 
        #           the line of code you need to do this. 
        #x = Lambda(one_hot)(out)
        
    # Step 3: Create model instance with the correct "inputs" and "outputs" (≈1 line)
    inference_model = Model([x0, a0], outputs)
    
    ### END CODE HERE ###
    
    return inference_model

In [112]:
inference_model = inference_model(LSTM_cell, densor, transfer_densor, n_a = 128, n_values =678)

ValueError: Layer lstm_2 expects 7 inputs, but it received 3 input tensors. Input received: [<tf.Tensor 'input_25:0' shape=(?, 1, 678) dtype=float32>, <tf.Tensor 'dense_4_5/Tanh:0' shape=(?, 128) dtype=float32>, <tf.Tensor 'dense_4_5/Tanh:0' shape=(?, 128) dtype=float32>]

In [59]:
a0_cap = train_features["1000092795"]
a0_cap

array([[3.8965435 , 3.8461769 , 0.        , ..., 0.        , 0.63524294,
        1.1605144 ]], dtype=float32)

In [71]:
X_cap = np.zeros((1, vocab_size))
X_cap[0][1] = 1
X_cap = X_cap.reshape(1, 1, vocab_size)
X_cap.shape

(1, 1, 678)

In [ ]:
def predict_and_sample(inference_model, x_initializer = x_initializer, a_initializer = a_initializer):

    # Use your inference model to predict an output sequence given x_initializer, a_initializer and c_initializer.
    pred = inference_model.predict([x_initializer, a_initializer])

    # Convert "pred" into an np.array() of indices with the maximum probabilities
    indices = np.argmax(np.array(pred), axis = 2)

    # Convert indices to one-hot vectors, the shape of the results should be (1, )
    results = to_categorical(indices)

    return results, indices

In [73]:
results, indices = predict_and_sample(inference_model, )

[array([[5.21047294e-01, 2.92173028e-02, 2.02262048e-02, 7.57777411e-03,
         1.15708550e-02, 9.15471651e-03, 1.44877415e-02, 1.60948541e-02,
         1.04367696e-02, 1.10839708e-02, 5.47612645e-03, 1.00016417e-02,
         2.85232207e-03, 3.97960749e-03, 7.43575767e-03, 5.75860217e-03,
         3.99200665e-03, 3.96529678e-03, 3.22219892e-03, 4.09858860e-03,
         2.80400645e-03, 2.48949369e-03, 5.75613696e-03, 3.73917096e-03,
         4.74457350e-03, 4.19977680e-03, 1.74152630e-03, 2.62591592e-03,
         1.93186977e-03, 2.47030426e-03, 3.16932518e-03, 2.41376995e-03,
         2.32212595e-03, 1.94749143e-03, 4.35352745e-03, 2.32403097e-03,
         4.11373936e-03, 2.34527071e-03, 2.58587557e-03, 2.16995296e-03,
         2.38124607e-03, 1.90077920e-03, 1.57156761e-03, 1.56494882e-03,
         2.18815077e-03, 2.96852505e-03, 1.98497926e-03, 1.64287316e-03,
         2.62150331e-03, 2.15858012e-03, 2.50260276e-03, 1.36215659e-03,
         2.02476350e-03, 2.01378413e-03, 1.08993053

In [74]:
desc = generate_desc(inference_model, tokenizer, a0_cap, max_length(train_descriptions))

ValueError: Error when checking input: expected input_3 to have 3 dimensions, but got array with shape (1, 4096)

In [36]:
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
    # seed the generation process
    in_text = 'startseq'
    # iterate over the whole length of the sequence
    for i in range(max_length):
        # integer encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # pad input
        sequence = pad_sequences([sequence], maxlen=max_length)
        # predict next word
        yhat = model.predict([photo,sequence], verbose=0)
        # convert probability to integer
        yhat = np.argmax(yhat)
        # map integer to word
        word = word_for_id(yhat, tokenizer)
        # stop if we cannot map the word
        if word is None:
            break
        # append as input for generating the next word
        in_text += ' ' + word
        # stop if we predict the end of the sequence
        if word == 'endseq':
            break
    return in_text